In [1]:
import sqlite3
import pprint
import pandas as pd

In [2]:
pd.set_option("display.max_columns",999)

In [3]:
db = sqlite3.connect('soccerdb.sqlite')

In [4]:
c = db.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('sqlite_sequence',), ('Player_Attributes',), ('Player',), ('Match',), ('League',), ('Country',), ('Team',), ('Team_Attributes',)]


In [5]:
#list((c.execute("SELECT * FROM Player LIMIT 5")))

# Baseball Questions - Part ii

# See all tables

In [6]:
# sqlquery = '''  SELECT *
#                 FROM sqlite_sequence
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM Player_Attributes
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM Player
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM Match
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM League
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM Country
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM Team
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM Team_Attributes
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

1. Which team scored the most points when playing at home?  

In [7]:
sqlquery = '''  SELECT SUM(home_team_goal) AS goals, team_long_name AS team
                FROM Match AS m
                JOIN Team AS t
                On t.team_api_id = m.home_team_api_id
                GROUP BY team
                ORDER BY goals DESC
                LIMIT 1;'''
            
pd.read_sql(sqlquery, db)

,goals,team
0,505,Real Madrid CF


2. Did this team also score the most points when playing away?  

In [8]:
sqlquery = '''  SELECT SUM(away_team_goal) AS goals, team_long_name AS team
                FROM Match AS m
                JOIN Team AS t
                On t.team_api_id = m.away_team_api_id
                GROUP BY team
                ORDER BY goals DESC
                LIMIT 1;'''
            
pd.read_sql(sqlquery, db)

,goals,team
0,354,FC Barcelona


3. How many matches resulted in a tie?  

In [9]:
sqlquery = '''  SELECT COUNT() AS Draws
                FROM Match
                WHERE home_team_goal=away_team_goal;'''
            
pd.read_sql(sqlquery, db)

,Draws
0,6596


4. How many players have Smith for their last name? How many have 'smith' anywhere in their name?

In [10]:
# Smith for last name
sqlquery = '''  SELECT COUNT(player_name)
                FROM Player
                WHERE player_name LIKE '%smith';
                '''
            
pd.read_sql(sqlquery, db)

,COUNT(player_name)
0,18


In [11]:
# Smith anywhere in name
sqlquery = '''  SELECT COUNT(player_name)
                FROM Player
                WHERE player_name LIKE '%smith';
                '''
            
pd.read_sql(sqlquery, db)

,COUNT(player_name)
0,18


5. What was the median tie score? Use the value determined in the previous question for the number of tie games. *Hint:* PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the [`WITH`](https://www.postgresql.org/docs/8.4/static/queries-with.html) command to store stepwise results as a table and then operate on these results. 

In [12]:
# a median is the middle result when results are listed in 
# in order by ASC or DECS

# There are 6596 draws
# So I want the 3298 RESULT from all the 0-0, 1-1, 2-2, 3-3, etc

In [13]:
6596/2

3298.0

In [14]:
sqlquery = '''  WITH num_draws AS
                       (SELECT Count() AS Draws
                       FROM Match
                       WHERE home_team_goal=away_team_goal),
                     last_is_median AS
                       (SELECT home_team_goal, away_team_goal
                       FROM Match
                       WHERE home_team_goal=away_team_goal
                       ORDER BY home_team_goal
                       LIMIT(SELECT * FROM num_draws)/2)
                SELECT last_is_median.home_team_goal, last_is_median.away_team_goal
                FROM last_is_median
                ORDER BY last_is_median.home_team_goal DESC
                LIMIT 1;
                '''
            
pd.read_sql(sqlquery, db)

,home_team_goal,away_team_goal
0,1,1


6. What percentage of players prefer their left or right foot? *Hint:* Calculate either the right or left foot, whichever is easier based on how you setup the problem.

In [119]:
# It appears some players prefer different feet at different points in time.
# This query returns each of their "apperances" in the table as a different player.
# Thus 24.31% of player "snapshots" have players preferring left feet.

sqlquery = '''
    SELECT COUNT(player_fifa_api_id) * 100.0 / (SELECT COUNT(player_fifa_api_id) FROM Player_Attributes) AS PercentLeft
    FROM Player_Attributes
    WHERE preferred_foot = 'left';   
            '''
pd.read_sql(sqlquery, db)

,PercentLeft
0,24.31432
